In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd

# Install and import kerastuner
!pip install -q -U keras-tuner
import keras_tuner as kt

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.4 MB/s eta 0:00:00


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'], inplace=True)
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.

column_count = len(application_df.columns)
unique_counts_list = []
column_names_list = list(application_df.columns.values)
values_list = []

for i in range(column_count):
  values = application_df.iloc[:,i].unique()
  count = len(values)
  unique_counts_list.append(count)
  values_list.append(values)

unique_counts_df = pd.DataFrame([unique_counts_list,values_list],columns=column_names_list)
unique_counts_df.head(15)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,17,6,71,5,4,2,9,2,8747,2
1,"[T10, T3, T5, T7, T4, T6, T2, T9, T19, T8, T13...","[Independent, CompanySponsored, Family/Parent,...","[C1000, C2000, C3000, C1200, C2700, C7000, C72...","[ProductDev, Preservation, Heathcare, Communit...","[Association, Co-operative, Trust, Corporation]","[1, 0]","[0, 1-9999, 10000-24999, 100000-499999, 10M-50...","[N, Y]","[5000, 108590, 6692, 142590, 31452, 7508025, 9...","[1, 0]"


In [4]:
# Look at APPLICATION_TYPE value counts for binning

app_type_value_counts_df = pd.DataFrame(application_df['APPLICATION_TYPE'].value_counts())
app_type_value_counts_df.head(20)


,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156
T13,66


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

all_app_types_descending = app_type_value_counts_df.index
print(all_app_types_descending)

application_types_to_replace = all_app_types_descending[12:17]
print(application_types_to_replace)


Index(['T3', 'T4', 'T6', 'T5', 'T19', 'T8', 'T7', 'T10', 'T9', 'T13', 'T12',
       'T2', 'T25', 'T14', 'T29', 'T15', 'T17'],
      dtype='object')
Index(['T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')


In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning

classification_value_counts_df = pd.DataFrame(application_df['CLASSIFICATION'].value_counts())
classification_value_counts_df.head(50)


,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116
C1270,114


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Dropping classifications with <30 values each, which means keeping the top 20 classifications

all_classifications_descending = classification_value_counts_df.index
print(all_classifications_descending)

total_classifications_count = len(all_classifications_descending)

classifications_to_replace = all_classifications_descending[45:total_classifications_count]
print(classifications_to_replace)


Index(['C1000', 'C2000', 'C1200', 'C3000', 'C2100', 'C7000', 'C1700', 'C4000',
       'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230',
       'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800',
       'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210',
       'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0',
       'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600',
       'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700',
       'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732',
       'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150'],
      dtype='object')
Index(['C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='ob

In [9]:

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
Other       26
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

# First auto-optimized model / Second overall NN model

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_application_df = pd.get_dummies(application_df, dtype=int)
encoded_application_df.head(50)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
5,1,5000,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
6,1,31452,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0
7,1,7508025,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,1,0
8,1,94389,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
9,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
y = encoded_application_df['IS_SUCCESSFUL']
X = encoded_application_df.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
# Create a method that creates a new Sequential model with hyperparameter options

X_dimensions_count = len(X.columns.values)

def create_model(hp):
    nn_auto_opt = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_auto_opt.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=X_dimensions_count))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_auto_opt.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_auto_opt.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_auto_opt.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_auto_opt


In [14]:
# Create and run kerastuner to optimize hyperparameters

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))


Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.7292128205299377

Best val_accuracy So Far: 0.7338775396347046
Total elapsed time: 00h 29m 26s


In [38]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 5,
 'num_layers': 4,
 'units_0': 3,
 'units_1': 5,
 'units_2': 7,
 'units_3': 5,
 'units_4': 3,
 'units_5': 5,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [15]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5732 - accuracy: 0.7339 - 750ms/epoch - 3ms/step
Loss: 0.573165237903595, Accuracy: 0.7338775396347046


# Second auto-optimized model / Third overall NN model

In [16]:
# Copy initial input dataframe (post-binning) to new instance for additional edits/optimizations
application_df2 = application_df.copy()

application_df2.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Drop income amount column/feature as potentially-'confusing' to see if it helps improve model accuracy/performance
application_df2.drop(columns=['INCOME_AMT'], inplace=True)

application_df2.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,N,142590,1


In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_application_df2 = pd.get_dummies(application_df2, dtype=int)
encoded_application_df2.head(50)


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,1,0
5,1,5000,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
6,1,31452,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
7,1,7508025,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,1,0
8,1,94389,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,0
9,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [19]:
# Split our preprocessed data into our features and target arrays
y2 = encoded_application_df2['IS_SUCCESSFUL']
X2 = encoded_application_df2.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=1)


In [20]:
# Create a StandardScaler instance
scaler2 = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler2.fit(X_train2)

# Scale the data
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)


In [21]:
# Create a method that creates a new Sequential model with hyperparameter options

X_dimensions_count2 = len(X2.columns.values)

def create_model(hp):
    nn_auto_opt2 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_auto_opt2.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=X_dimensions_count2))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_auto_opt2.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_auto_opt2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_auto_opt2.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_auto_opt2


In [ ]:
# Create and run kerastuner to optimize hyperparameters

tuner2 = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

tuner2.search(X_train_scaled2,y_train2,epochs=20,validation_data=(X_test_scaled2,y_test2))


Trial 60 Complete [00h 01m 00s]
val_accuracy: 0.7286297082901001

Best val_accuracy So Far: 0.7286297082901001
Total elapsed time: 00h 32m 17s


In [ ]:
# Get best model hyperparameters
best_hyper2 = tuner2.get_best_hyperparameters(1)[0]
best_hyper2.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 1,
 'units_0': 5,
 'units_1': 1,
 'units_2': 3,
 'units_3': 3,
 'units_4': 5,
 'units_5': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [ ]:
# Evaluate best model against full test data
best_model2 = tuner2.get_best_models(1)[0]
model_loss2, model_accuracy2 = best_model2.evaluate(X_test_scaled2,y_test2,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 1s - loss: 0.5651 - accuracy: 0.7286 - 879ms/epoch - 3ms/step
Loss: 0.5651049017906189, Accuracy: 0.7286297082901001


# Third auto-optimized model / Fourth overall NN model

In [11]:
# Copy initial input dataframe (post-binning) to new instance for additional edits/optimizations
application_df3 = application_df.copy()

application_df3.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [12]:
# Drop potentially-'confusing' columns with especially unbalanced distributions of values
application_df3.drop(columns=['STATUS','SPECIAL_CONSIDERATIONS'], inplace=True)

application_df3.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_application_df3 = pd.get_dummies(application_df3, dtype=int)
encoded_application_df3.head(50)


,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
5,5000,1,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
6,31452,1,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
7,7508025,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
8,94389,1,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
9,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [14]:
# Split our preprocessed data into our features and target arrays
y3 = encoded_application_df3['IS_SUCCESSFUL']
X3 = encoded_application_df3.drop(columns=['IS_SUCCESSFUL'])

# Split the preprocessed data into a training and testing dataset
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state=1)


In [15]:
# Create a StandardScaler instance
scaler3 = StandardScaler()

# Fit the StandardScaler
X_scaler3 = scaler3.fit(X_train3)

# Scale the data
X_train_scaled3 = X_scaler3.transform(X_train3)
X_test_scaled3 = X_scaler3.transform(X_test3)


In [18]:
# Create a method that creates a new Sequential model with hyperparameter options

X_dimensions_count3 = len(X3.columns.values)

def create_model(hp):
    nn_auto_opt3 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_auto_opt3.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=X_dimensions_count3))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_auto_opt3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_auto_opt3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_auto_opt3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_auto_opt3


In [19]:
# Create and run kerastuner to optimize hyperparameters

tuner3 = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

tuner3.search(X_train_scaled3,y_train3,epochs=20,validation_data=(X_test_scaled3,y_test3))


Trial 60 Complete [00h 01m 23s]
val_accuracy: 0.731195330619812

Best val_accuracy So Far: 0.7342274188995361
Total elapsed time: 00h 25m 45s


In [20]:
# Get best model hyperparameters
best_hyper3 = tuner3.get_best_hyperparameters(1)[0]
best_hyper3.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 5,
 'units_0': 9,
 'units_1': 5,
 'units_2': 5,
 'units_3': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'units_4': 1,
 'tuner/trial_id': '0015'}

In [21]:
# Evaluate best model against full test data
best_model3 = tuner3.get_best_models(1)[0]
model_loss3, model_accuracy3 = best_model3.evaluate(X_test_scaled3,y_test3,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

268/268 - 1s - loss: 0.5775 - accuracy: 0.7342 - 591ms/epoch - 2ms/step
Loss: 0.5775216817855835, Accuracy: 0.7342274188995361


In [22]:
# Export our model to HDF5 file
best_model3.save('AlphabetSoupCharity_Optimization.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
from google.colab import files
files.download('AlphabetSoupCharity_Optimization.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>